In [1]:
import numpy as np
from tqdm import tqdm

import mlrfit as mf
import numba as nb

from scipy.linalg import block_diag  
from mlrfit import frob_loss, rel_diff, diag_sparseBCt, LinOpResidualMatrix, compute_perm_residual

import mfmodel as mfm

In [2]:
n = 800
signal_to_noise = 4


nsamples = 40
L = 5


ranks = np.array([10, 7, 5, 4, 1])
rank = ranks.sum()

pi_rows = np.random.permutation(n)
hpart = {'rows':{'pi':pi_rows, 'lk':[]}, 'cols':{'pi':pi_rows, 'lk':[]}} 
for ngroups in [2, 5, 9, 17, n+1]:
       hpart['rows']['lk'] += [ np.linspace(0, n, ngroups, endpoint=True, dtype=int)]
hpart['rows']['lk'][1] = np.delete(hpart['rows']['lk'][1], -2)
hpart['rows']['lk'][2] = np.delete(hpart['rows']['lk'][2], -4)
hpart['cols']['lk'] = hpart['rows']['lk']
part_sizes = mfm.print_hpart_numgroups(hpart)
mfm.valid_hpart(hpart)

true_mlr, true_sparse_F, true_D_noise = mfm.generate_mlr_model(n, hpart, ranks, signal_to_noise=signal_to_noise)
C = mfm.generate_data(true_sparse_F, true_D_noise, nsamples, true_mlr)
Z = (C - C.mean(axis=1, keepdims=True))[hpart["rows"]["pi"], :]
del C
unpermuted_A = (Z @ Z.T / (Z.shape[1]-1))[true_mlr.pi_inv_rows, :][:, true_mlr.pi_inv_cols]
# permute to put clusters on diagonal
Y = Z.T
N = Y.shape[0]

level=0, num_groups=1, mean_size=800.0
level=1, num_groups=3, mean_size=266.7
level=2, num_groups=7, mean_size=114.3
level=3, num_groups=16, mean_size=50.0
level=4, num_groups=800, mean_size=1.0
signal_var=26.19073360423691, noise_var=6.1901710742440805
SNR=4.231019351502368, signal_to_noise=4


In [3]:
hat_A1 = mf.MLRMatrix()
hat_A1.hpart = hpart
PSD = True

B, C = hat_A1.init_B_C(ranks, hpart, init_type='zeros')
hat_A1.B, hat_A1.C = B, C
losses = hat_A1.factor_fit((Y[:, true_mlr.pi_inv_rows].T/np.sqrt(N), 
                            Y[:, true_mlr.pi_inv_rows].T/np.sqrt(N)), ranks, hat_A1.hpart, PSD=PSD, freq=10, 
                          eps_ff=1e-6, printing=True, max_iters_ff=30, symm=True, warm_start=True)

itr=0, [], [10  7  5  4  1]
itr=0, 0.3553672288904666, [10  7  5  4  1], time_v_epoch=2.4362599849700928, time_loss=0.01446080207824707
itr=10, 0.3465971782882264, [10  7  5  4  1], time_v_epoch=0.795828104019165, time_loss=0.014462947845458984
itr=20, 0.3456512527861053, [10  7  5  4  1], time_v_epoch=0.8528141975402832, time_loss=0.019987821578979492


In [4]:
hat_A2 = mf.MLRMatrix()
hat_A2.hpart = hpart

B, C = hat_A2.init_B_C(ranks, hpart, init_type='zeros')
hat_A2.B, hat_A2.C = B, C
cov_matrix = ((Y.T/np.sqrt(N)) @ (Y/np.sqrt(N)))[true_mlr.pi_inv_rows, :][:, true_mlr.pi_inv_cols]
losses2 = hat_A2.factor_fit(cov_matrix, ranks, hat_A2.hpart, PSD=PSD, freq=10, 
                          eps_ff=1e-6, printing=True, max_iters_ff=30, symm=True, warm_start=True)

itr=0, [], [10  7  5  4  1]


/Users/parshakova.tanya/Documents/projects/MLR_Matrices/mlr_fitting/mlrfit/utils.py:409: NumbaPerformanceWarning: np.dot() is faster on contiguous arrays, called on (Array(float64, 2, 'A', False, aligned=True), Array(float64, 2, 'A', False, aligned=True))
  hat_A_except_level[r1:r2, c1:c2] += np.dot(B_level[r1:r2], C_level[c1:c2].T)


itr=0, 0.3553672288904666, [10  7  5  4  1], time_v_epoch=2.04154109954834, time_loss=0.5203440189361572
itr=10, 0.34659717828822634, [10  7  5  4  1], time_v_epoch=0.27061009407043457, time_loss=0.004147052764892578
itr=20, 0.3456512527861054, [10  7  5  4  1], time_v_epoch=0.23865199089050293, time_loss=0.01857900619506836


In [8]:
assert np.allclose(hat_A1.matrix(), hat_A2.matrix())


num_levels = len(hpart['rows']['lk']) 
for level in range(num_levels):
    A_l1 = hat_A1._block_diag_BCt(level, hpart, hat_A1.B[:,ranks[:level].sum():ranks[:level+1].sum()], \
                                                hat_A1.C[:,ranks[:level].sum():ranks[:level+1].sum()])
    A_l2 = hat_A2._block_diag_BCt(level, hpart, hat_A2.B[:,ranks[:level].sum():ranks[:level+1].sum()], \
                                                hat_A2.C[:,ranks[:level].sum():ranks[:level+1].sum()])
    assert np.allclose(A_l1, A_l2)

assert np.allclose(np.array(losses), np.array(losses2))

print("PASSED")

PASSED
